In [2]:
!pip install tensorflow

     |████████████████████████████████| 489.6 MB 46 kB/s s eta 0:00:01    |█████████▌                      | 144.9 MB 9.8 MB/s eta 0:00:36     |███████████▋                    | 178.3 MB 8.5 MB/s eta 0:00:37     |██████████████                  | 214.8 MB 9.9 MB/s eta 0:00:28
     |████████████████████████████████| 5.8 MB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 4.1 MB 8.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 6.0 MB/s eta 0:00:011
     |████████████████████████████████| 2.1 MB 8.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 9.4 MB/s eta 0:00:01
     |████████████████████████████████| 13.3 MB 9.4 MB/s eta 0:00:01
     |████████████████████████████████| 463 kB 9.4 MB/s eta 0:00:01
     |████████████████████████████████| 126 

In [30]:
# 학습
# Data : ALL

import tensorflow as tf
import h5py
import numpy as np
import os
from PIL import Image
import cv2

print(os.getcwd())


/home/vividlee/Desktop/YeloCar


In [5]:
dataset_version = 'deep4pc_test1'
model_version = 'model_' + dataset_version

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

# model = tf.keras.models.load_model('./script/Model/'+model_version)
# print(model.evaluate(x_test, y_test))
# model.summary()

# model = tf.saved_model.load('./Model/Model_' + dataset_version + '/' + model_version)
# outputs = model(x_test[0:10])

# print(outputs)

# print(x_test[0:10])

2022-01-18 16:46:59.418362: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-18 16:46:59.418479: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-01-18 16:46:59.418545: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-01-18 16:46:59.445869: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-01-18 16:46:59.446023: W tensorflow/stream_executor/platform/default/dso_loader

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 6, 32)         0

2022-01-18 16:54:42.334409: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./Model/Model_deep4pc_test1/model_deep4pc_test1/assets


## dataset : 'deep4pc_test1'
### Epoch 12/100 673/673 [==============================] - 113s 168ms/step - loss: 0.0038 - mae: 0.0413 - mse: 0.0038 - val_loss: 0.0123 - val_mae: 0.0777 - val_mse: 0.0123

In [6]:
dataset_version = 'deep4pc_test1'
model_version = 'model_' + dataset_version + '_2'

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

# model = tf.keras.models.load_model('./script/Model/'+model_version)
# print(model.evaluate(x_test, y_test))
# model.summary()

# model = tf.saved_model.load('./Model/Model_' + dataset_version + '/' + model_version)
# outputs = model(x_test[0:10])

# print(outputs)

# print(x_test[0:10])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 23, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 10, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 4, 6, 32)        

2022-01-18 16:57:42.329926: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6347022336 exceeds 10% of free system memory.


Epoch 1/100
673/673 [==============================] - 114s 169ms/step - loss: 0.0078 - mae: 0.0582 - mse: 0.0078 - val_loss: 0.0116 - val_mae: 0.0737 - val_mse: 0.0116
Epoch 2/100
673/673 [==============================] - 111s 166ms/step - loss: 0.0061 - mae: 0.0507 - mse: 0.0061 - val_loss: 0.0113 - val_mae: 0.0745 - val_mse: 0.0113
Epoch 3/100
673/673 [==============================] - 112s 166ms/step - loss: 0.0056 - mae: 0.0491 - mse: 0.0056 - val_loss: 0.0127 - val_mae: 0.0772 - val_mse: 0.0127
Epoch 4/100
673/673 [==============================] - 111s 166ms/step - loss: 0.0053 - mae: 0.0477 - mse: 0.0053 - val_loss: 0.0121 - val_mae: 0.0761 - val_mse: 0.0121
Epoch 5/100
673/673 [==============================] - 111s 165ms/step - loss: 0.0050 - mae: 0.0466 - mse: 0.0050 - val_loss: 0.0123 - val_mae: 0.0762 - val_mse: 0.0123
Epoch 6/100
673/673 [==============================] - 112s 166ms/step - loss: 0.0048 - mae: 0.0456 - mse: 0.0048 - val_loss: 0.0122 - val_mae: 0.0765 - va

In [11]:
from PIL import Image
import cv2

In [28]:
np.set_printoptions(threshold=np.inf, linewidth=np.inf)

model = tf.saved_model.load('./Model/Model_' + dataset_version + '/' + model_version)

x_tested = x_test[0:10]
# print(type(x_test[0:10]))
# print(x_test[0:10].shape)
# print(type(x_tested))
# print(x_tested.shape)

outputs = model(x_tested)

print(outputs)

x_tested *= 255

for i in range (0, 10) :
    cv2.imwrite('filename_' + str(i) + '.jpeg', x_tested[i])


tf.Tensor(
[[0.4722335  0.1559705 ]
 [0.47223353 0.15597047]
 [0.47223586 0.15597549]
 [0.47223628 0.15597016]
 [0.4722375  0.15596777]
 [0.4722398  0.15596272]
 [0.4722401  0.15596005]
 [0.47224066 0.15596175]
 [0.472234   0.15597227]
 [0.4722414  0.15596355]], shape=(10, 2), dtype=float32)


In [29]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [31]:
dataset_version = 'deep4pc_test1'

model_version = 'model_' + dataset_version + '_3_patience10'

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version + '/' + model_version)
#model.summary()
outputs = model(x_test[0:10])

print(outputs)

# x_test *= 255

createFolder('./Test/' + model_version)    
for i in range (0, 10) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 23, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 10, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 4, 6, 32)        

2022-01-18 18:40:03.627758: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6347022336 exceeds 10% of free system memory.


Epoch 1/100
673/673 [==============================] - 121s 179ms/step - loss: 0.0079 - mae: 0.0583 - mse: 0.0079 - val_loss: 0.0122 - val_mae: 0.0776 - val_mse: 0.0122
Epoch 2/100
673/673 [==============================] - 113s 168ms/step - loss: 0.0061 - mae: 0.0506 - mse: 0.0061 - val_loss: 0.0118 - val_mae: 0.0766 - val_mse: 0.0118
Epoch 3/100
673/673 [==============================] - 113s 168ms/step - loss: 0.0056 - mae: 0.0488 - mse: 0.0056 - val_loss: 0.0132 - val_mae: 0.0802 - val_mse: 0.0132
Epoch 4/100
673/673 [==============================] - 114s 170ms/step - loss: 0.0052 - mae: 0.0475 - mse: 0.0052 - val_loss: 0.0129 - val_mae: 0.0785 - val_mse: 0.0129
Epoch 5/100
673/673 [==============================] - 113s 169ms/step - loss: 0.0049 - mae: 0.0462 - mse: 0.0049 - val_loss: 0.0126 - val_mae: 0.0771 - val_mse: 0.0126
Epoch 6/100
673/673 [==============================] - 113s 168ms/step - loss: 0.0047 - mae: 0.0452 - mse: 0.0047 - val_loss: 0.0119 - val_mae: 0.0750 - va

OSError: No file or directory found at ./script/Model/model_deep4pc_test1_3_patience15

In [39]:
dataset_version = 'deep4pc_test1'
model_version = 'model_' + dataset_version + '_4_patience15'

createFolder('./Test/' + model_version)
f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version + '/' + model_version)
#model.summary()
outputs = model(x_test[0:10])

print(outputs)
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 10) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 23, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 10, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 4, 6, 32)        

2022-01-18 19:34:06.316865: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6347022336 exceeds 10% of free system memory.


Epoch 1/100
673/673 [==============================] - 114s 169ms/step - loss: 0.0080 - mae: 0.0591 - mse: 0.0080 - val_loss: 0.0113 - val_mae: 0.0724 - val_mse: 0.0113
Epoch 2/100
673/673 [==============================] - 115s 170ms/step - loss: 0.0061 - mae: 0.0508 - mse: 0.0061 - val_loss: 0.0121 - val_mae: 0.0744 - val_mse: 0.0121
Epoch 3/100
673/673 [==============================] - 116s 172ms/step - loss: 0.0056 - mae: 0.0491 - mse: 0.0056 - val_loss: 0.0126 - val_mae: 0.0770 - val_mse: 0.0126
Epoch 4/100
673/673 [==============================] - 114s 169ms/step - loss: 0.0053 - mae: 0.0478 - mse: 0.0053 - val_loss: 0.0136 - val_mae: 0.0819 - val_mse: 0.0136
Epoch 5/100
673/673 [==============================] - 115s 170ms/step - loss: 0.0050 - mae: 0.0466 - mse: 0.0050 - val_loss: 0.0136 - val_mae: 0.0812 - val_mse: 0.0136
Epoch 6/100
673/673 [==============================] - 113s 168ms/step - loss: 0.0048 - mae: 0.0456 - mse: 0.0048 - val_loss: 0.0123 - val_mae: 0.0760 - va

In [40]:
dataset_version = 'deep4pc_test1'
model_version = 'model_' + dataset_version + '_5_patience20'

createFolder('./Test/' + model_version)
f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version + '/' + model_version)
#model.summary()
outputs = model(x_test[0:10])

print(outputs)
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 10) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 23, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 10, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 4, 6, 32)        